In [1]:
from keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

2022-09-28 20:01:09.775320: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-28 20:01:09.900075: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-28 20:01:09.936828: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-28 20:01:10.690289: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [ ]:
# references:
# https://www.datasnips.com/41/tensorflow-multiclass-image-classification-using-image-data-generator/

In [2]:

src_path_train = "../full_data_split/train/"
src_path_val = '../full_data_split/val/'
src_path_test = "../full_data_split/test/"

train_datagen = ImageDataGenerator(
        #rescale=1 / 255.0,
        #rotation_range=20,
        #zoom_range=0.05,
        #width_shift_range=0.05,
        #height_shift_range=0.05,
        #shear_range=0.05,
        #horizontal_flip=True,
        #fill_mode="nearest",
        #validation_split=0.20
        )

val_datagen = ImageDataGenerator()

test_datagen = ImageDataGenerator(
                #rescale=1 / 255.0
                    )

In [3]:
batch_size = 32
img_height = 180
img_width = 180

In [11]:
batch_size = 8
train_generator = train_datagen.flow_from_directory(
    directory=src_path_train,
    #target_size=(100, 100),
    #color_mode="rgb",
    batch_size=batch_size,
    #class_mode="categorical",
    class_mode='sparse',
    #subset='training',
    shuffle=True,
    seed=42
)
valid_generator = val_datagen.flow_from_directory(
    directory=src_path_val,
    #target_size=(100, 100),
    #color_mode="rgb",
    batch_size=batch_size,
    #class_mode="categorical",
    class_mode='sparse',
    #subset='validation',
    shuffle=True,
    seed=42
)
test_generator = test_datagen.flow_from_directory(
    directory=src_path_test,
    #target_size=(100, 100),
    #color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

Found 651174 images belonging to 9812 classes.
Found 141160 images belonging to 9812 classes.
Found 145082 images belonging to 9812 classes.


In [12]:
class_names = train_generator.labels # classes # train_generator.class_names
#print(class_names)

num_classes = train_generator.num_classes # len(class_names)

print(class_names, num_classes)

[   0    0    0 ... 9811 9811 9811] 9812


In [13]:
# simple CNN with resize layer followed by rescaling in the second layer
# 
model = Sequential([
    layers.Resizing(height=img_height, width=img_width, 
                    input_shape=(None, None, 3)
                   ),
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3) ), 
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [14]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_1 (Resizing)       (None, 180, 180, 3)       0         
                                                                 
 rescaling_1 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 90, 90, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                  

In [17]:
# ~ 20 min/epoch using g4dn.4xlarge (1x T4 GPU)
# try the same with g4dn.12xlarge (4x T4, 48 CPU but seems CPUs wasted?)
# try p3.2xlarge (1x V100, 8 vCPU) --> from core counts, seems 1 V100 == 2 T4
model.fit(train_generator, validation_data=valid_generator,
                    steps_per_epoch = train_generator.n // train_generator.batch_size,
                    validation_steps = valid_generator.n // valid_generator.batch_size,
                    epochs=5
                   )

Epoch 1/5
 4680/81396 [>.............................] - ETA: 17:45 - loss: 9.2778 - accuracy: 8.0128e-05

/home/ubuntu/miniconda3/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:845: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))


81396/81396 [==============================] - 1389s 17ms/step - loss: 9.2562 - accuracy: 7.3714e-05 - val_loss: 9.2246 - val_accuracy: 1.0626e-04
Epoch 2/5
81396/81396 [==============================] - 988s 12ms/step - loss: 9.2439 - accuracy: 7.3714e-05 - val_loss: 9.2223 - val_accuracy: 9.9178e-05
Epoch 3/5
81396/81396 [==============================] - 1011s 12ms/step - loss: 9.2425 - accuracy: 5.5285e-05 - val_loss: 9.2220 - val_accuracy: 1.0626e-04
Epoch 4/5
 4905/81396 [>.............................] - ETA: 12:34 - loss: 9.2254 - accuracy: 5.0968e-05

KeyboardInterrupt: 